In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse

from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

from xgboost import XGBRFRegressor as XGBRF
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.linear_model import LogisticRegression as LogReg

import lazypredict
from lazypredict.Supervised import LazyClassifier as Lc
from lazypredict.Supervised import LazyRegressor as Lr
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR
from sklearn.linear_model import GammaRegressor as GR

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
salaries= pd.read_csv('../data/salaries_data.csv')

test = pd.read_csv('../data/testeo.csv')

In [3]:
#Borramos columnas que no hace falta
test.drop('salary', inplace=True, axis=1)
#test.drop('salary_currency', inplace=True, axis=1)

In [4]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [5]:
def diction(col):
    lista = list(set(list(salaries[col].unique()) + list(test[col].unique())))
    nums =[i for i in range(len(lista))]
    print({(lista[i]): (nums[i]) for i in range(len(lista))})
    return {(lista[i]): (nums[i]) for i in range(len(lista))}

dictio_experience_level = diction('experience_level')
dictio_employment_type = diction('employment_type')
dictio_job_title = diction('job_title')
dictio_employee_residence = diction('employee_residence')
dictio_company_location = diction('company_location')
dictio_company_size = diction('company_size')
dictio_salary_currency = diction('salary_currency')

{'MI': 0, 'EN': 1, 'SE': 2, 'EX': 3}
{'PT': 0, 'FL': 1, 'FT': 2, 'CT': 3}
{'Data Science Manager': 0, 'NLP Engineer': 1, 'Data Engineer': 2, 'Data Analytics Manager': 3, 'Big Data Engineer': 4, 'Research Scientist': 5, 'Principal Data Scientist': 6, 'Machine Learning Scientist': 7, 'Data Analyst': 8, 'AI Scientist': 9, 'BI Data Analyst': 10, 'Cloud Data Engineer': 11, 'Data Analytics Lead': 12, 'ETL Developer': 13, 'Machine Learning Manager': 14, 'Staff Data Scientist': 15, 'Principal Data Engineer': 16, 'ML Engineer': 17, 'Data Analytics Engineer': 18, 'Lead Machine Learning Engineer': 19, 'Data Scientist': 20, 'Principal Data Analyst': 21, 'Data Architect': 22, 'Machine Learning Engineer': 23, 'Director of Data Engineering': 24, 'Finance Data Analyst': 25, 'Computer Vision Software Engineer': 26, 'Head of Machine Learning': 27, 'Head of Data Science': 28, 'Computer Vision Engineer': 29, 'Financial Data Analyst': 30, 'Head of Data': 31, 'Lead Data Engineer': 32, 'Applied Machine Learn

In [6]:
salaries_copy_1 = salaries.copy(deep=True)
salaries_copy_1.head()

salaries_copy_1.drop(['salary'], inplace=True,axis=1)
print(salaries_copy_1.head())

test_copy_1 = test.copy(deep=True)
print(test_copy_1.head())

   work_year experience_level employment_type        job_title  \
0       2022               SE              FT    Data Engineer   
1       2022               SE              FT    Data Engineer   
2       2021               MI              FT  BI Data Analyst   
3       2021               MI              CT      ML Engineer   
4       2021               MI              FT    Data Engineer   

  salary_currency  salary_in_usd employee_residence  remote_ratio  \
0             USD         140250                 US           100   
1             USD         135000                 US           100   
2             USD         100000                 US           100   
3             USD         270000                 US           100   
4             EUR          26005                 RO             0   

  company_location company_size  
0               US            M  
1               US            M  
2               US            M  
3               US            L  
4               US

In [7]:
test_copy_1.shape, salaries_copy_1.shape

((107, 9), (500, 10))

In [8]:
test_copy_1['y'] = -1
test_copy_1.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y
0,2020,SE,FT,Machine Learning Scientist,USD,JP,0,JP,S,-1
1,2020,MI,FT,Lead Data Analyst,USD,US,100,US,L,-1
2,2020,MI,FT,Data Analyst,USD,US,100,US,L,-1
3,2020,MI,FT,Machine Learning Engineer,CNY,CN,0,CN,M,-1
4,2020,MI,FT,Product Data Analyst,INR,IN,100,IN,L,-1


In [9]:
test_copy_1.shape, salaries_copy_1.shape

((107, 10), (500, 10))

In [10]:
todo = pd.concat([test_copy_1, salaries_copy_1])

In [11]:
todo.job_title = todo.job_title.apply(lambda x: dictio_job_title[x])
todo.employment_type = todo.employment_type.apply(lambda x: dictio_employment_type[x])
todo.employee_residence = todo.employee_residence.apply(lambda x: dictio_employee_residence[x])
todo.experience_level = todo.experience_level.apply(lambda x: dictio_experience_level[x])
todo.company_size = todo.company_size.apply(lambda x: dictio_company_size[x])
todo.company_location = todo.company_location.apply(lambda x: dictio_company_location[x])
todo.salary_currency = todo.salary_currency.apply(lambda x: dictio_salary_currency[x])
todo.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,2,2,7,6,10,0,9,0,-1.00,NaN
1,2020,0,2,44,6,32,100,28,2,-1.00,NaN
2,2020,0,2,8,6,32,100,28,2,-1.00,NaN
3,2020,0,2,23,10,26,0,20,1,-1.00,NaN
4,2020,0,2,45,11,33,100,29,2,-1.00,NaN


In [12]:
ok = todo.loc[todo['y'] == -1]
ok.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,2,2,7,6,10,0,9,0,-1.00,NaN
1,2020,0,2,44,6,32,100,28,2,-1.00,NaN
2,2020,0,2,8,6,32,100,28,2,-1.00,NaN
3,2020,0,2,23,10,26,0,20,1,-1.00,NaN
4,2020,0,2,45,11,33,100,29,2,-1.00,NaN


In [13]:
ok.shape

(107, 11)

In [14]:
ok_s = todo.loc[todo['y'] != -1]
ok_s.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2022,2,2,2,6,32,100,28,1,NaN,140250.00
1,2022,2,2,2,6,32,100,28,1,NaN,135000.00
2,2021,0,2,10,6,32,100,28,1,NaN,100000.00
3,2021,0,3,17,6,32,100,28,2,NaN,270000.00
4,2021,0,2,2,1,8,0,28,2,NaN,26005.00


In [15]:
ok_s.shape

(500, 11)

In [16]:
ok_s.drop('y', axis=1, inplace=True)
ok.drop('salary_in_usd', axis=1, inplace=True)
ok.drop('y', axis=1, inplace=True)

## Modelos

In [17]:
X = ok_s.drop('salary_in_usd', axis = 1)
y = ok_s.salary_in_usd

In [19]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.99, test_size=0.01, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((495, 9), (5, 9), (495,), (5,))

In [20]:
lc = Lc(predictions=True)
models, predictions = lc.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  2.96it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
GaussianNB,0.20,0.12,None,0.27,0.02
AdaBoostClassifier,0.00,0.00,None,0.00,0.21
LinearSVC,0.00,0.00,None,0.00,0.81
SVC,0.00,0.00,None,0.00,0.18
SGDClassifier,0.00,0.00,None,0.00,0.17
RidgeClassifierCV,0.00,0.00,None,0.00,0.02
RidgeClassifier,0.00,0.00,None,0.00,0.01
RandomForestClassifier,0.00,0.00,None,0.00,0.19
Perceptron,0.00,0.00,None,0.00,0.10


In [21]:
lr = Lr(predictions=True)
models, predictions = lr.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:02<00:00, 17.96it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
MLPRegressor,31.65,-37.31,107638.04,0.29
LinearSVR,31.44,-37.05,107263.98,0.00
KernelRidge,30.65,-36.06,105863.30,0.01
AdaBoostRegressor,7.52,-7.15,49641.76,0.02
BaggingRegressor,4.35,-3.19,35582.80,0.02
KNeighborsRegressor,3.75,-2.44,32257.99,0.01
RandomForestRegressor,3.72,-2.40,32085.33,0.13
DecisionTreeRegressor,3.56,-2.20,31107.05,0.01
ExtraTreeRegressor,3.56,-2.20,31107.05,0.01


In [22]:
#Inicializamos modelos
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
dtr=DTR()
lgbmr=LGBMR()
xgbrf = XGBRF()
linreg = LinReg()
knr = KNR()
logreg = LogReg()
hgbr = HGBR()
gr = GR()

#Entrenamientos
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train, verbose=0)
dtr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)
xgbrf.fit(X_train, y_train)
linreg.fit(X_train, y_train)
knr.fit(X_train, y_train)
logreg.fit(X_train, y_train)
hgbr.fit(X_train, y_train)
gr.fit(X_train, y_train)

#Predicciones
y_pred_rfr = rfr.predict(X_test)
y_pred_etr = etr.predict(X_test)
y_pred_gbr = gbr.predict(X_test)
y_pred_xgbr = xgbr.predict(X_test)
y_pred_ctr = ctr.predict(X_test)
y_pred_dtr = dtr.predict(X_test)
y_pred_lgbmr = lgbmr.predict(X_test)
y_pred_xgbrf = xgbrf.predict(X_test)
y_pred_linreg = linreg.predict(X_test)
y_pred_knr = knr.predict(X_test)
y_pred_logreg = logreg.predict(X_test)
y_pred_hgbr = hgbr.predict(X_test)
y_pred_gr = gr.predict(X_test)

#Calculo de error
print(mse(y_test, y_pred_rfr, squared=False), mse(y_test, y_pred_etr, squared=False), mse(y_test, y_pred_gbr, squared=False), 
mse(y_test, y_pred_xgbr, squared=False), mse(y_test, y_pred_ctr, squared=False), mse(y_test, y_pred_dtr, squared=False),
mse(y_test, y_pred_lgbmr, squared=False), mse(y_test, y_pred_xgbrf, squared=False), mse(y_test, y_pred_linreg, squared=False),
mse(y_test, y_pred_knr, squared=False), mse(y_test, y_pred_logreg, squared=False), mse(y_test, y_pred_hgbr, squared=False),
mse(y_test, y_pred_gr, squared=False))

32456.999151840686 31107.05042561317 27217.0661467236 31089.167938994713 30258.72555627629 31107.05042561317 29495.801550722106 31237.495747952602 20760.665843943145 29727.39375754289 38348.715360491544 30822.372667268643 14279.841204188862


### Mejor Modelo: GammaRegressor

In [23]:
gr.fit(X_train, y_train)

y_pred_gr = gr.predict(ok)
y_pred_gr[:10]

len(y_pred_gr)

107

## Exportar

In [24]:
res = pd.read_csv('../data/muestra.csv')
res.salary_in_usd = y_pred_gr
res.head()

,id,salary_in_usd
0,0,82078.31
1,1,112539.84
2,2,103175.22
3,3,78903.93
4,4,107503.82


In [25]:
res.to_csv('../data/resultado_gr_contodo_curren.csv', index = False)